## Vizaulization TODO:
* come up with idea on who to visualize categorical attributes. Should discretization be done beforehand?
* find out who can one obtain useful knowledge from date attributes, how to visualize it?
* analyze and iterpret numerical attributes


In [1]:
import pandas as pd
import pandas_profiling as pdp
import os
import numpy as np
import re
import plotly.express as px
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
df =pd.read_csv('data/ltfs3_train.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128655 entries, 0 to 128654
Data columns (total 26 columns):
ID                 128655 non-null int64
Frequency          128655 non-null object
InstlmentMode      128655 non-null object
LoanStatus         128655 non-null object
PaymentMode        128655 non-null object
BranchID           128655 non-null int64
Area               117002 non-null object
Tenure             128655 non-null int64
AssetCost          128655 non-null int64
AmountFinance      128655 non-null float64
DisbursalAmount    128655 non-null float64
EMI                128655 non-null float64
DisbursalDate      128655 non-null object
MaturityDAte       128654 non-null object
AuthDate           128655 non-null object
AssetID            128655 non-null int64
ManufacturerID     128655 non-null int64
SupplierID         128655 non-null int64
LTV                128655 non-null float64
SEX                128596 non-null object
AGE                128596 non-null float64
MonthlyIn

In [4]:
cols = df.columns
num_cols = df._get_numeric_data().columns
categorical_cols = list(set(cols) - set(num_cols))
categorical_cols = [i for i in categorical_cols if re.match('^(?!(TOP-UP MONTH|.*DATE)$).*$',i.upper())]
print(categorical_cols)

['PaymentMode', 'SEX', 'LoanStatus', 'Frequency', 'Area', 'City', 'InstlmentMode', 'State']


In [10]:
def draw_plots_categorical(grouped_by,target,df):
    temp = pd.DataFrame()
    unique = pd.unique(df[grouped_by].dropna())
    for attr in unique:
        temp[attr] = df[df[grouped_by]==attr][target].value_counts()
    columns = dict(zip(list(range(len(temp.columns))),temp.columns))
    temp = temp.rename(columns=columns)
    return get_plots(temp,grouped_by,target)


In [11]:
def get_plots_categorical(temp,attribute,target):
    fig = make_subplots(rows=2, cols=2, column_widths=[0.8, 0.3],
                    specs=[[ {"colspan": 2},{}],
                        [{"type": "xy"}, {"type": "pie"}]])
    for counter,i in enumerate(temp.index):
        fig.add_trace(go.Bar(y=temp[temp.index==i].values[0],x=temp.columns,name=i,
                         legendgroup='group1',marker_color=px.colors.qualitative.Prism[counter]), row=1, col=1)
    pie_temp = temp.sum()
    temp = temp.T
    for counter,i in enumerate(temp.index):
        fig.add_trace(go.Bar(y=temp[temp.index==i].values[0],x=temp.columns,name=i,
                    legendgroup='group1',showlegend=False,marker_color=px.colors.qualitative.Prism[counter%2]), row=2, col=1)
    fig.add_trace(go.Pie(labels=pie_temp.index,values=pie_temp,legendgroup='group2',marker=dict(colors= px.colors.qualitative.Prism)),row =2,col=2)
    fig.layout.update(barmode='group',
        height=800, legend=dict(tracegroupgap = 250),title=f'{attribute} vizualization based on {target}')
    fig.show()

In [12]:
draw_plots_categorical('LoanStatus','Top-up Month',df)

In [13]:
draw_plots_categorical('Frequency','Top-up Month',df)

In [14]:
draw_plots_categorical('PaymentMode','Top-up Month',df)